In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import yaml
import nibabel as nib

In [2]:
with open(Path("./config/config.yaml"), "r") as f:
    config = yaml.safe_load(f)

path_output = Path(config["path_output"]).absolute()
path_atlas = Path(config["path_atlas"]).absolute()
path_parcellation = Path(config["path_parcellation"]).absolute()
path_labels = Path(config["path_labels"]).absolute()
path_trans_seg = path_output / "transformed_seg"

In [3]:
def load_labels(label_file, index_col=0, description_col=1):
    df = pd.read_csv(label_file, header=None, delimiter=" ")
    # df = pd.read_excel(label_file)
    labels = dict(zip(df.iloc[:, index_col], df.iloc[:, description_col]))
    return labels

In [4]:
labels = load_labels(path_labels)

In [5]:
atlas = nib.load(path_atlas)
atlas_data = atlas.get_fdata().astype(int)

In [6]:
def analyze_tumor_locations(tumor_seg_path, atlas_data, labels_dict):
    tumor_img = nib.load(tumor_seg_path)
    tumor_data = tumor_img.get_fdata().astype(int)

    tumor_mask = (tumor_data > 0).astype(bool)
    total_tumor_voxels = np.sum(tumor_mask)

    if total_tumor_voxels == 0:
        return {}

    location_counts = {}

    for label_value, region_name in labels_dict.items():
        region_mask = atlas_data == label_value
        overlap = np.sum(tumor_mask & region_mask)
        if overlap > 0:
            percentage = (overlap / total_tumor_voxels) * 100
            location_counts[region_name] = percentage

    return location_counts

In [7]:
results = []

if path_trans_seg.exists():
    for tumor_file in path_trans_seg.glob("*.nii.gz"):
        tumor_name = tumor_file.stem.replace(".nii", "")

        locations = analyze_tumor_locations(tumor_file, atlas_data, labels)

        results.append(
            {
                "tumor_file": tumor_name,
                "locations": locations,
            }
        )
else:
    print(f"Transformed segmentation directory not found: {path_trans_seg}")

In [8]:
df = pd.DataFrame(
    [
        {"tumor_file": r["tumor_file"], "location": loc, "percentage": pct}
        for r in results
        for loc, pct in r["locations"].items()
    ]
)

In [9]:
df.sort_values(by="percentage", ascending=False).head(10)

,tumor_file,location,percentage
141,C1072314_305_SegmWT,ctx-rh-lingual,6.999004
142,C1072314_305_SegmWT,ctx-rh-medialorbitofrontal,6.700082
140,C1072314_305_SegmWT,ctx-rh-lateralorbitofrontal,6.235092
59,C1031970_225_SegmWT,ctx-lh-transversetemporal,4.868340
57,C1031970_225_SegmWT,ctx-lh-frontalpole,4.717105
58,C1031970_225_SegmWT,ctx-lh-temporalpole,4.701392
60,C1031970_225_SegmWT,ctx-lh-insula,4.656872
139,C1072314_305_SegmWT,ctx-rh-lateraloccipital,4.589511
61,C1031970_225_SegmWT,ctx-rh-bankssts,4.187453
56,C1031970_225_SegmWT,ctx-lh-supramarginal,4.138351


In [10]:
pivot = df.pivot(index="tumor_file", columns="location", values="percentage").fillna(0)

In [11]:
analysis_output = path_output / "analysis_output"
analysis_output.mkdir(parents=True, exist_ok=True)

pivot.to_csv(analysis_output / "locations.csv")

print(f"Results saved to {analysis_output}")

Results saved to c:\Users\m\Documents\run-synthmorph\output\analysis_output
